# Simple Example
To illustrate how to use GluonTS, we train a DeepAR-model and make predictions using the airpassengers dataset. The dataset consists of a single time series of monthly passenger numbers between 1949 and 1960. We train the model on the first nine years and make predictions for the remaining three years.


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from gluonts.torch import DeepAREstimator

# Load data from a CSV file into a PandasDataset
df = pd.read_csv(
    "https://raw.githubusercontent.com/AileenNielsen/"
    "TimeSeriesAnalysisWithPython/master/data/AirPassengers.csv",
    index_col=0,
    parse_dates=True,
)
dataset = PandasDataset(df, target="#Passengers")

# Split the data for training and testing
training_data, test_gen = split(dataset, offset=-36)
#split?
print(f"training_data: {(training_data)}")
training_data_iterable=iter(training_data)
print(next(training_data_iterable))
#print(next(training_data_iterable))
test_data = test_gen.generate_instances(prediction_length=12, windows=3)
print(f"test_data: {test_data}")


In [ ]:
# Train the model and make predictions
model = DeepAREstimator(
    prediction_length=12, freq="M", trainer_kwargs={"max_epochs": 5}
).train(training_data)

forecasts = list(model.predict(test_data.input))

# Plot predictions
plt.plot(df["1954":], color="black")
fc_count = 0
for forecast in forecasts:
    print(forecast)
    fc_count += 1
    forecast.plot()
print(f"fc_count:{fc_count}")
plt.legend(["True values"], loc="upper left", fontsize="xx-large")
plt.show()

In [ ]:
# Plot predictions
# forecasts contain the number of windows
plt.figure()
plt.plot(df["1954":], color="black")
fc_count = 0
for forecast in forecasts:
    print(len(forecast.samples))
    fc_count += 1
    forecast.plot()
    if fc_count > 0:
        break
print(f"fc_count:{fc_count}")
plt.legend(["True values"], loc="upper left", fontsize="xx-large")
plt.show()